In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import librosa

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data exploration 

In [5]:
train_audio_path = '/kaggle/input/birdclef-2024/train_audio'

data = []

for bird_species in os.listdir(train_audio_path) :
    species_folder = os.path.join(train_audio_path, bird_species)
    
    if os.path.isdir(species_folder) :
        for audio_file in os.listdir(species_folder) :
            if audio_file.endswith('.ogg') :
                audio_path = os.path.join(species_folder, audio_file)
                
                signal, sr = librosa.load(audio_path, sr= None)
                
                mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
                mfccs_mean = np.mean(mfccs.T, axis=0)
                
                data.append(
                {
                    'species': bird_species,
                    'file': audio_file,
                    'mfcc': mfccs_mean
                }
                )
                
df = pd.DataFrame(data)

print(df.head())

   species          file                                               mfcc
0  redspu1  XC312771.ogg  [-437.41815, 46.508083, -78.70597, 82.640465, ...
1  redspu1   XC37584.ogg  [-276.8721, 72.655945, -124.2829, -5.833173, -...
2  redspu1  XC283501.ogg  [-243.03755, 52.037975, -71.31738, 16.098063, ...
3  redspu1  XC469479.ogg  [-212.04, 38.166824, -30.160025, 15.724985, 3....
4  redspu1  XC471046.ogg  [-531.05646, 57.274635, 5.7300725, 15.054413, ...
